## Model 1.0
Powered by [Eleonora Priori](https://www.est-en.unito.it/do/docenti.pl/Alias?eleonora.priori#tab-profilo) and [Pietro Terna](https://terna.to.it/) 


In [1]:
%%javascript
// to avoid scroll in windows
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

====================================================================================================

## 1

import libs \
MPI init \
context and runner definition \
t(), T(), Tc() function definitions \
random number generator rng creation \
initialization of the parameters from yaml file\
memory allocations to manage ghosts


====================================================================================================

In [2]:
import time
from mpi4py import MPI
from repast4py import context as ctx
import repast4py 
from repast4py import parameters
from repast4py import schedule
from repast4py import core
from typing import Tuple, List, Dict
import numpy as np
import csv


comm = MPI.COMM_WORLD
rank    = comm.Get_rank()
rankNum = comm.Get_size() 

# create the context to hold the agents and manage cross process
# synchronization
context = ctx.SharedContext(comm)

# Initialize the default schedule runner, HERE to create the t() function,
# returning the tick value
runner = schedule.init_schedule_runner(comm)

# tick number
def t():
    return int(runner.schedule.tick)

#Initializes the repast4py.parameters.params dictionary with the model input parameters.
params = parameters.init_params("model1.yaml", "")


#generate random seed
repast4py.random.init(rng_seed=params['myRandom.seed'][rank]) #each rank has a seed
rng = repast4py.random.default_rng 


#timer T()
startTime=-1
def T():
    global startTime
    if startTime < 0:
        startTime=time.time()
    return time.time() - startTime
T()

#cpuTimer Tc()
startCpuTime=-1
def Tc():
    global startCpuTime
    if startCpuTime < 0:
        startCpuTime=time.process_time()
    return time.process_time() - startCpuTime
Tc()

agent_cache={} # dict with uid as keys and agents' tuples as values

===================================================================================================

## 2

create agents' classes and restore_agent function 



===================================================================================================

In [3]:
class Firm(core.Agent):

    TYPE = 0
    
    def __init__(self, local_id: int, rank: int, labor:int, capital:float,\
                 duration:int, recipe: float, laborProductivity: float, dimensionalClass: int):
        super().__init__(id=local_id, type=Firm.TYPE, rank=rank) #uid
        self.labor=labor
        self.capital=capital
        self.duration=duration
        self.recipe = recipe
        self.laborProductivity=laborProductivity
        self.dimensionalClass=dimensionalClass
        self.lostProduction=0
        self.inventories=0
        self.inProgressInventories=0
        self.appRepository=[] #aPP=aProductiveProcess
        self.availableLabor=self.labor
        self.availableCapital=self.capital
        
        self.profits=[]
        self.revenues=[]
        self.totalCosts=[]
        self.addedValue=[]
        self.finalInventories=[]
        self.initialInventories=[]
        
    def produce(self, productionOrder:float)->tuple:
        
        self.costOfProductionOrder=0
        output=0
        costOfUnusedFactors=0
        totalLostProduction=0
        totalCostOfLostProduction=0
        
        
        lastOrderProductionInEachPeriod=productionOrder/self.duration
        requiredLabor=np.ceil(lastOrderProductionInEachPeriod/self.laborProductivity)
        requiredCapital=requiredLabor*self.recipe
        
        #TMP TMP TMP
        #print(self.uid,"last in",productionOrder,productionOrder/self.duration,\
               #requiredLabor,requiredCapital)

        #create a new aPP or skip the order
        if requiredLabor <= self.labor and requiredCapital <= self.capital: 
            aProductiveProcess = ProductiveProcess(lastOrderProductionInEachPeriod, \
                                                   requiredLabor, requiredCapital, self.duration)
            self.appRepository.append(aProductiveProcess)
        
        for aProductiveProcess in self.appRepository:  
            
            if aProductiveProcess.hasResources or \
            (self.availableLabor >= aProductiveProcess.requiredLabor and\
                self.availableCapital >= aProductiveProcess.requiredCapital):
                
                if not aProductiveProcess.hasResources:
                    self.availableLabor -= aProductiveProcess.requiredLabor
                    self.availableCapital -= aProductiveProcess.requiredCapital
                    aProductiveProcess.hasResources = True 
                #production
                (outputOfThePeriod, requiredLabor, requiredCapital,\
                lostProduction, costOfLostProduction)=aProductiveProcess.step(productionOrder)               
                     
                output += outputOfThePeriod
                
                cost = requiredLabor*params['wage'] + requiredCapital*params['costOfCapital']
                self.costOfProductionOrder += cost
                
                totalLostProduction += lostProduction
                totalCostOfLostProduction += costOfLostProduction
              
        
                if aProductiveProcess.failure:
                    self.inProgressInventories -= cost*(aProductiveProcess.productionClock-1)
                
                else:
                    if aProductiveProcess.productionClock < aProductiveProcess.duration:
                        self.inProgressInventories += cost
                    else:
                        self.inventories+=cost*self.duration
                        self.inProgressInventories -= cost*(self.duration-1)

        costOfUnusedFactors =  self.availableLabor*params['wage'] + \
                                self.availableCapital*params['costOfCapital']
                             
        
        # remove concluded aPPs from the list
        for i in range(len(self.appRepository)-1,-1,-1):
            if self.appRepository[i].productionClock == self.appRepository[i].duration: 
                self.availableLabor+=self.appRepository[i].requiredLabor
                self.availableCapital+=self.appRepository[i].requiredCapital
                #self.appRepository.remove(aProductiveProcess)
                del self.appRepository[i]

        
        #print(self.uid,"\n", productionOrder,  self.labor, self.capital, self.recipe,\
              #self.laborProductivity,self.duration,"\n",\
              #output, costOfProductionOrder, costOfUnusedFactors, self.inventories,\
              #self.inProgressInventories, totalLostProduction,totalCostOfLostProduction, flush=True)
        
        return(output, self.costOfProductionOrder, costOfUnusedFactors,self.inventories,\
               self.inProgressInventories, totalLostProduction, totalCostOfLostProduction)
    
    def makeBalancesheet(self):
        if t()==0: self.initialInventories.append(0)
        else: self.initialInventories.append(self.finalInventories[-1])
        
        self.totalCosts.append(self.costOfProductionOrder)
        self.finalInventories.append(self.inventories+self.inProgressInventories)
        self.profits.append(self.finalInventories[-1]-self.totalCosts[-1]-self.initialInventories[-1])
        self.addedValue.append(self.profits[-1]+self.totalCosts[-1])
        
        print(self, "init inventories = ", self.initialInventories[-1], ", total cost = ", self.totalCosts[-1], \
             ", final inventories = ", self.finalInventories[-1], ", profits = ", self.profits[-1], ", added value = ",\
             self.addedValue[-1])

    
###########################################################################################################################


class ProductiveProcess():
    def __init__(self, targetProductionOfThePeriod:float, requiredLabor:int, requiredCapital:float, duration:int):
        
        self.targetProductionOfThePeriod=targetProductionOfThePeriod
        self.requiredLabor = requiredLabor
        self.requiredCapital = requiredCapital
        self.duration = duration
        self.productionClock=0
        self.hasResources= False
        
    def step(self, productionOrder)->tuple:
        
        lostProduction=0
        self.costOfLostProduction=0
        self.productionClock += 1
        self.failure=False
        
        # production failure
        if params['probabilityToFailProductionChoices'] >= rng.random():
            self.failure=True
            #print("failure",flush=True)
            lostProduction=self.targetProductionOfThePeriod*self.productionClock
            self.targetProductionOfThePeriod=0
            self.costOfLostProduction=(params['wage']* self.requiredLabor+\
                                       params['costOfCapital']* self.requiredCapital)*self.productionClock
            self.duration = self.productionClock   

        return(self.targetProductionOfThePeriod, self.requiredLabor, self.requiredCapital, \
               lostProduction, self.costOfLostProduction)

===================================================================================================

## 3

the model

===================================================================================================

In [4]:
class Model:
    
    global params
    PARAMS = params
    
    def __init__(self, params: Dict):
        
        self.totalProduction=[]
        self.totalCostOfProduction=[]
        self.totalCostOfUnusedFactors=[]
        self.totalInventories=[]
        self.totalInProgressInventories=[]
        self.totalLostProduction=[]
        self.totalCostOfLostProduction=[]
        
        #the context and the runner are created in step 1 
      
        runner.schedule_event(          0.0,     self.initGhosts) 
        runner.schedule_repeating_event(0.0,  1, self.counter)
        runner.schedule_repeating_event(0.1,  1, self.firmsProducing)
        runner.schedule_repeating_event(0.2,  1, self.enterprisesMakingBalancesheet) #enterprises=firms+banks
        
        runner.schedule_stop(params['howManyCycles'])
        runner.schedule_end_event(self.finish)
        
        ####################################################################################################
        ###################################### CREATE FIRM AGENTS ##########################################
        ####################################################################################################
        
        #importing csv file containing info about firms 
        #share of firms of that class, L-min, L-max, K-min, K-max, t-min, t-max, recipe, L-prod
        with open('firm-features.csv', newline='') as csvfile:
            firmReader= csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
            
            self.rowNumber=0
            k=0
            #for each record in firm-features.csv
            for row in firmReader:
                print(row)
                for i in range(int(row[0] * params['Firm.count'])// rankNum):
                    labor= rng.integers(row[1], row[2]+1)
                    capital= row[3] + rng.random()*(row[4] -row[3])
                    duration= rng.integers(row[5], row[6]+1)
                    recipe= row[7] #K/L 
                    laborProductivity= row[8]
                    dimensionalClass=self.rowNumber
                    aFirm =Firm(k, rank, labor, capital, duration, recipe, laborProductivity, dimensionalClass)
                    context.add(aFirm)
                    k += 1
                self.rowNumber += 1

        
    #initialize ghosts by sending them in the ranks before starting the simulation
    def initGhosts(self):
        pass
    
    #count the cycles number
    def counter(self):
        if int(t()) % params["tickNumber.betweenChecks"] == 0: 
            print("rank", rank, "tick", t(), flush=True)
            
  
    
    def firmsProducing(self):
        self.totalProduction.append([0]*(self.rowNumber)) #for each cycle adds a sub-list of lenght number of dimClass
        self.totalCostOfProduction.append([0]*(self.rowNumber))
        self.totalCostOfUnusedFactors.append([0]*(self.rowNumber))
        self.totalInventories.append([0]*(self.rowNumber))
        self.totalInProgressInventories.append([0]*(self.rowNumber))
        self.totalLostProduction.append([0]*(self.rowNumber))
        self.totalCostOfLostProduction.append([0]*(self.rowNumber))
        
        for aFirm in context.agents(agent_type=0):
            #print(aFirm.dimensionalClass)
            tupleOfProductionResults = aFirm.produce(rng.random()*params['maxOrderProduction'][aFirm.dimensionalClass])
            self.totalProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[0]
            self.totalCostOfProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[1]
            self.totalCostOfUnusedFactors[t()][aFirm.dimensionalClass] += tupleOfProductionResults[2]
            self.totalInventories[t()][aFirm.dimensionalClass] += tupleOfProductionResults[3]
            self.totalInProgressInventories[t()][aFirm.dimensionalClass] += tupleOfProductionResults[4] 
            self.totalLostProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[5]
            self.totalCostOfLostProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[6]
            
    def enterprisesMakingBalancesheet(self):
        for aFirm in context.agents(agent_type=0):
            aFirm.makeBalancesheet()
            
    #finish
    def finish(self):
        
        # infos for data_analysis.ipynb
        with open('plotInfo.csv', 'w', newline='')\
          as file:
            writer = csv.writer(file)
            writer.writerow((params["log_file_root"],rankNum))
        
        
        print("\n total production", self.totalProduction, flush=True)
        print("\n total cost of production", self.totalCostOfProduction, flush=True)
        print("\n total cost of unused factors", self.totalCostOfUnusedFactors, flush=True)
        print("\n total inventories", self.totalInventories, flush=True)
        print("\n total in progress inventories", self.totalInProgressInventories, flush=True)
        print("\n total lost production", self.totalLostProduction, flush=True)
        print("\n total cost of lost production", self.totalCostOfLostProduction, flush=True)

        print("THE END!", flush=True)
        
        names=["_total_production_","_total_cost_of_production_","_total_cost_of_unused_factors_",
               "_total_inventories_","_total_in_progress_inventories_",
               "_total_lost_production_","_total_cost_of_lost_production_"]
        contents=[self.totalProduction,self.totalCostOfProduction,
                  self.totalCostOfUnusedFactors,
                  self.totalInventories,self.totalInProgressInventories,
                  self.totalLostProduction,self.totalCostOfLostProduction]
        
        for s in range(len(names)):
            with open(params["log_file_root"]+names[s]+str(rank)+'.csv', 'w',\
                  newline='') as file:
                writer = csv.writer(file)
                for k in range(params["howManyCycles"]):
                    writer.writerow(contents[s][k])
    
    def start(self):
        runner.execute()

=========================================================================================================

## 4

run the model

==========================================================================================================

In [5]:
def run(params: Dict):
    
    model = Model(params) 
    model.start()
    
run(params)

[0.1, 60.0, 80.0, 200.0, 400.0, 5.0, 10.0, 5.0, 0.8]
[0.3, 30.0, 50.0, 50.0, 100.0, 2.0, 4.0, 2.0, 0.7]
[0.6, 10.0, 20.0, 5.0, 10.0, 1.0, 1.0, 0.5, 0.6]
rank 0 tick 0
Agent(0, 0, 0) init inventories =  0 , total cost =  33.0 , final inventories =  33.0 , profits =  0.0 , added value =  33.0
Agent(1, 0, 0) init inventories =  0 , total cost =  12.0 , final inventories =  0.0 , profits =  -12.0 , added value =  0.0
Agent(2, 0, 0) init inventories =  0 , total cost =  28.8 , final inventories =  28.8 , profits =  0.0 , added value =  28.8
Agent(3, 0, 0) init inventories =  0 , total cost =  18.0 , final inventories =  18.0 , profits =  0.0 , added value =  18.0
Agent(4, 0, 0) init inventories =  0 , total cost =  5.25 , final inventories =  5.25 , profits =  0.0 , added value =  5.25
Agent(5, 0, 0) init inventories =  0 , total cost =  0 , final inventories =  0 , profits =  0 , added value =  0
Agent(6, 0, 0) init inventories =  0 , total cost =  9.45 , final inventories =  9.45 , profit

Agent(0, 0, 0) init inventories =  238.5 , total cost =  39.0 , final inventories =  187.5 , profits =  -90.0 , added value =  -51.0
Agent(1, 0, 0) init inventories =  136.8 , total cost =  30.0 , final inventories =  166.79999999999998 , profits =  -2.842170943040401e-14 , added value =  29.99999999999997
Agent(2, 0, 0) init inventories =  224.39999999999998 , total cost =  42.0 , final inventories =  266.4 , profits =  0.0 , added value =  42.0
Agent(3, 0, 0) init inventories =  177.59999999999997 , total cost =  37.2 , final inventories =  200.39999999999998 , profits =  -14.399999999999977 , added value =  22.800000000000026
Agent(4, 0, 0) init inventories =  32.55 , total cost =  9.45 , final inventories =  42.0 , profits =  0.0 , added value =  9.45
Agent(5, 0, 0) init inventories =  39.9 , total cost =  0 , final inventories =  39.9 , profits =  0.0 , added value =  0.0
Agent(6, 0, 0) init inventories =  49.35 , total cost =  10.5 , final inventories =  59.85 , profits =  0.0 , 

Agent(0, 0, 0) init inventories =  462.0 , total cost =  91.5 , final inventories =  553.5 , profits =  0.0 , added value =  91.5
Agent(1, 0, 0) init inventories =  319.2 , total cost =  26.400000000000002 , final inventories =  342.00000000000006 , profits =  -3.599999999999909 , added value =  22.800000000000093
Agent(2, 0, 0) init inventories =  482.4 , total cost =  49.2 , final inventories =  531.6 , profits =  5.684341886080802e-14 , added value =  49.20000000000006
Agent(3, 0, 0) init inventories =  326.4 , total cost =  27.6 , final inventories =  354.0 , profits =  0.0 , added value =  27.6
Agent(4, 0, 0) init inventories =  59.849999999999994 , total cost =  12.6 , final inventories =  72.44999999999999 , profits =  -7.105427357601002e-15 , added value =  12.599999999999993
Agent(5, 0, 0) init inventories =  76.64999999999999 , total cost =  2.1 , final inventories =  78.74999999999999 , profits =  0.0 , added value =  2.1
Agent(6, 0, 0) init inventories =  85.05 , total cost

Agent(4, 0, 0) init inventories =  86.09999999999998 , total cost =  7.35 , final inventories =  93.44999999999997 , profits =  0.0 , added value =  7.35
Agent(5, 0, 0) init inventories =  97.64999999999999 , total cost =  8.4 , final inventories =  106.05 , profits =  0.0 , added value =  8.4
Agent(6, 0, 0) init inventories =  114.45 , total cost =  13.65 , final inventories =  128.1 , profits =  -1.4210854715202004e-14 , added value =  13.649999999999986
Agent(7, 0, 0) init inventories =  86.09999999999998 , total cost =  10.5 , final inventories =  96.59999999999998 , profits =  0.0 , added value =  10.5
Agent(8, 0, 0) init inventories =  52.5 , total cost =  10.5 , final inventories =  63.0 , profits =  0.0 , added value =  10.5
Agent(9, 0, 0) init inventories =  150.15 , total cost =  0 , final inventories =  150.15 , profits =  0.0 , added value =  0.0
rank 0 tick 18
Agent(0, 0, 0) init inventories =  924.0 , total cost =  81.0 , final inventories =  994.5 , profits =  -10.5 , ad

Agent(3, 0, 0) init inventories =  774.0 , total cost =  42.0 , final inventories =  816.0 , profits =  0.0 , added value =  42.0
Agent(4, 0, 0) init inventories =  131.24999999999997 , total cost =  6.3 , final inventories =  137.54999999999998 , profits =  0.0 , added value =  6.3
Agent(5, 0, 0) init inventories =  143.85 , total cost =  14.7 , final inventories =  158.54999999999998 , profits =  0.0 , added value =  14.7
Agent(6, 0, 0) init inventories =  160.65 , total cost =  2.1 , final inventories =  162.75 , profits =  0.0 , added value =  2.1
Agent(7, 0, 0) init inventories =  121.79999999999998 , total cost =  2.1 , final inventories =  123.89999999999998 , profits =  0.0 , added value =  2.1
Agent(8, 0, 0) init inventories =  85.05000000000001 , total cost =  5.25 , final inventories =  90.30000000000001 , profits =  0.0 , added value =  5.25
Agent(9, 0, 0) init inventories =  176.4 , total cost =  6.3 , final inventories =  182.70000000000002 , profits =  0.0 , added value 

...

Agent(5, 0, 0) init inventories =  616.3499999999999 , total cost =  0 , final inventories =  616.3499999999999 , profits =  0.0 , added value =  0.0
Agent(6, 0, 0) init inventories =  511.34999999999997 , total cost =  9.45 , final inventories =  520.8 , profits =  0.0 , added value =  9.45
Agent(7, 0, 0) init inventories =  343.35000000000014 , total cost =  7.35 , final inventories =  350.70000000000016 , profits =  0.0 , added value =  7.35
Agent(8, 0, 0) init inventories =  348.59999999999997 , total cost =  0 , final inventories =  348.59999999999997 , profits =  0.0 , added value =  0.0
Agent(9, 0, 0) init inventories =  684.6000000000003 , total cost =  2.1 , final inventories =  686.7000000000003 , profits =  0.0 , added value =  2.1
rank 0 tick 89
Agent(0, 0, 0) init inventories =  5745.0 , total cost =  79.5 , final inventories =  5742.0 , profits =  -82.5 , added value =  -3.0
Agent(1, 0, 0) init inventories =  2507.9999999999995 , total cost =  30.0 , final inventories =  

rank 0 tick 94
Agent(0, 0, 0) init inventories =  6100.5 , total cost =  106.5 , final inventories =  6057.0 , profits =  -150.0 , added value =  -43.5
Agent(1, 0, 0) init inventories =  2636.3999999999996 , total cost =  31.2 , final inventories =  2665.2 , profits =  -2.399999999999636 , added value =  28.800000000000363
Agent(2, 0, 0) init inventories =  3753.5999999999985 , total cost =  43.2 , final inventories =  3796.7999999999984 , profits =  0.0 , added value =  43.2
Agent(3, 0, 0) init inventories =  3715.2 , total cost =  48.0 , final inventories =  3763.2 , profits =  0.0 , added value =  48.0
Agent(4, 0, 0) init inventories =  490.35000000000014 , total cost =  3.15 , final inventories =  493.5000000000001 , profits =  0.0 , added value =  3.15
Agent(5, 0, 0) init inventories =  665.6999999999999 , total cost =  4.2 , final inventories =  669.9 , profits =  0.0 , added value =  4.2
Agent(6, 0, 0) init inventories =  541.7999999999998 , total cost =  2.1 , final inventories

Agent(3, 0, 0) init inventories =  3837.6 , total cost =  20.4 , final inventories =  3858.0 , profits =  0.0 , added value =  20.4
Agent(4, 0, 0) init inventories =  505.0500000000001 , total cost =  11.55 , final inventories =  516.6000000000001 , profits =  0.0 , added value =  11.55
Agent(5, 0, 0) init inventories =  705.6 , total cost =  7.35 , final inventories =  712.95 , profits =  0.0 , added value =  7.35
Agent(6, 0, 0) init inventories =  563.8499999999999 , total cost =  0 , final inventories =  563.8499999999999 , profits =  0.0 , added value =  0.0
Agent(7, 0, 0) init inventories =  435.7500000000002 , total cost =  1.05 , final inventories =  436.80000000000024 , profits =  0.0 , added value =  1.05
Agent(8, 0, 0) init inventories =  393.75 , total cost =  0 , final inventories =  393.75 , profits =  0.0 , added value =  0.0
Agent(9, 0, 0) init inventories =  733.9500000000003 , total cost =  8.4 , final inventories =  742.3500000000003 , profits =  0.0 , added value =  


 total cost of unused factors [[83.16838584878138, 107.56834970756523, 63.84066704047855], [71.16838584878138, 87.16834970756523, 58.59066704047855], [54.668385848781384, 73.96834970756522, 41.79066704047856], [32.168385848781384, 43.968349707565224, 68.04066704047855], [30.668385848781384, 46.368349707565216, 58.59066704047855], [60.668385848781384, 88.36834970756522, 57.54066704047856], [77.16838584878138, 57.16834970756522, 44.94066704047856], [87.66838584878138, 61.96834970756522, 32.34066704047856], [81.66838584878138, 60.768349707565214, 66.99066704047854], [74.16838584878138, 51.16834970756522, 60.690667040478544], [39.668385848781384, 34.36834970756522, 71.19066704047854], [23.168385848781384, 45.16834970756523, 55.44066704047856], [24.668385848781384, 63.16834970756523, 58.590667040478564], [30.668385848781384, 66.76834970756522, 64.89066704047856], [12.668385848781384, 47.56834970756522, 54.390667040478554], [24.668385848781384, 43.96834970756522, 55.44066704047856], [15.668


 total in progress inventories [[33.0, 46.8, 0.0], [78.0, 32.4, 0.0], [139.5, 100.79999999999998, 0.0], [223.5, 58.8, 0.0], [144.0, 69.6, 0.0], [73.5, 58.8, 0.0], [22.5, 75.6, 0.0], [51.0, 38.39999999999999, 0.0], [78.0, 129.59999999999997, 0.0], [105.0, 14.39999999999999, 0.0], [121.5, 124.79999999999998, 0.0], [154.5, 45.59999999999998, 0.0], [216.0, 76.79999999999998, 0.0], [256.5, 22.799999999999986, 0.0], [172.5, 141.6, 0.0], [121.5, 3.5999999999999894, 0.0], [169.5, 103.19999999999999, 0.0], [159.0, 59.999999999999986, 0.0], [229.5, 103.19999999999999, 0.0], [37.5, 28.79999999999998, 0.0], [94.5, 136.79999999999998, 0.0], [130.5, 1.1999999999999857, 0.0], [222.0, 119.99999999999999, 0.0], [243.0, 62.39999999999999, 0.0], [174.0, 109.19999999999999, 0.0], [189.0, 33.59999999999998, 0.0], [153.0, 145.2, 0.0], [210.0, 23.999999999999993, 0.0], [180.0, 104.4, 0.0], [97.5, 56.400000000000006, 0.0], [76.5, 86.4, 0.0], [30.0, 23.999999999999996, 0.0], [16.5, 125.99999999999999, 0.0], [